In [1]:
import tensorflow as tf

In [2]:
from utils_20191230 import *

In [3]:
tf.logging.set_verbosity(tf.logging.INFO)

In [4]:
# Load BERT Config
path = '../KorBERT/2_bert_download_002_bert_morp_tensorflow/002_bert_morp_tensorflow/'
FLAGS.bert_config_file = path + 'bert_config.json'
bert_config = BertConfig.from_json_file(FLAGS.bert_config_file)

In [5]:
if FLAGS.max_seq_length > bert_config.max_position_embeddings:
    raise ValueError(
        "Cannot use sequence length %d because the BERT model "
        "was only trained up to sequence length %d" %
        (FLAGS.max_seq_length, bert_config.max_position_embeddings))
else:
    print('     (FLAGS) MAX_SEQ_LENGTH           :', FLAGS.max_seq_length)
    print('(BERTConfig) MAX_POSITION_EMBEDDINGS  :', bert_config.max_position_embeddings)

     (FLAGS) MAX_SEQ_LENGTH           : 128
(BERTConfig) MAX_POSITION_EMBEDDINGS  : 512


In [6]:
# do not use tpu
tpu_cluster_resolver = None
if FLAGS.use_tpu and FLAGS.tpu_name:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
        FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)
else:
    print('Do not use TPU')

Do not use TPU


In [7]:
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps, # 1000
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop, # 1000
        num_shards=FLAGS.num_tpu_cores, # 8
        per_host_input_for_training=is_per_host)
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [8]:
# tensorflow gpu 사용 가능한지 체크
tf.test.is_gpu_available()

True

In [9]:
# 한국어 vocab 사전을 등록
FLAGS.vocab_file = path + 'vocab.korean_morp.list' 

In [10]:
import pandas as pd

dacon_path = '../dacon문자스미싱/filedown (2)/'
df_train = pd.read_csv(dacon_path + 'train.csv')
df_test = pd.read_csv(dacon_path + 'public_test.csv')

In [11]:
from sklearn.model_selection import train_test_split

df_train = df_train.set_index('id')
df_test = df_test.set_index('id')

X_train, X_valid, y_train, y_valid = train_test_split(
                 df_train[[col for col in df_train.columns if col != 'smishing']], 
                 df_train['smishing'],
                 random_state=42, test_size=.2,
                 stratify=df_train['smishing'])
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((236756, 2), (59189, 2), (236756,), (59189,))

In [12]:
df_train = pd.concat((X_train, y_train), axis=1)
df_valid = pd.concat((X_valid, y_valid), axis=1)

# sample 100개씩 뽑아서 미리 test
df_train.to_csv(dacon_path + 'train.tsv', sep='\t')
df_valid.to_csv(dacon_path + 'dev.tsv', sep='\t')
df_test.to_csv(dacon_path + 'test.tsv', sep='\t')

In [13]:
# DataProcessor 제작
class SmishingProcessor(DataProcessor):

    def get_train_examples(self, data_dir, filename='train.tsv'):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, filename)), "train")

    def get_dev_examples(self, data_dir, filename='dev.tsv'):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, filename)), "dev")

    def get_test_examples(self, data_dir, filename='test.tsv'):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, filename)), "test")

    def get_labels(self):
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = convert_to_unicode(line[2])
            if set_type == "test":
                label = "0"
            else:
                label = convert_to_unicode(line[-1])
            examples.append(
                InputExample(guid=guid, text_a=text_a, label=label))
        return examples

In [14]:
processor = SmishingProcessor()
label_list = processor.get_labels()

# get train samples
train_examples = processor.get_train_examples(dacon_path, 'train.tsv')
num_train_steps = int(
    len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

In [15]:
# record ETRI model weights
FLAGS.init_checkpoint = path + 'model.ckpt'

In [16]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),             # 2
    init_checkpoint=FLAGS.init_checkpoint,  # None
    learning_rate=FLAGS.learning_rate,      # 5e-05
    num_train_steps=num_train_steps,        # 22195
    num_warmup_steps=num_warmup_steps,      # 2219
    use_tpu=FLAGS.use_tpu,                  # False
    use_one_hot_embeddings=FLAGS.use_tpu)   # False

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,                        # False
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,      # 32
    eval_batch_size=FLAGS.eval_batch_size,        # 8
    predict_batch_size=FLAGS.predict_batch_size   # 8
)

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\jinma\\AppData\\Local\\Temp\\tmpp5e7nksj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015A19BB28D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, nu

In [17]:
FLAGS.output_dir = './output_dir/smishing/'
tf.gfile.MakeDirs(FLAGS.output_dir)

In [18]:
train_file = os.path.join(FLAGS.output_dir, 'train_nonspacing.tf_record')

In [19]:
from getpass import getpass

openapi_key = getpass(prompt='Password: ', stream=None)

Password: ········


In [20]:
openapi_key = None

In [21]:
tokenizer = BERTTokenizer(FLAGS.vocab_file, do_lower_case=True)

In [22]:
file_based_convert_examples_to_features(train_examples,
                                        label_list,
                                        FLAGS.max_seq_length,
                                        tokenizer,
                                        openapi_key,
                                        train_file)



INFO:tensorflow:Writing example 0 of 236756
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-1
INFO:tensorflow:tokens: [CLS] X X X/SL_ 고객/NNG_ 님/XSN_ ./SF_ X X X/SL_ [SEP]
INFO:tensorflow:input_ids: 2 3047 3047 1496 1291 1123 7 3047 3047 1496 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:Writing example 130000 of 236756
INFO:tensorflow:Writing example 140000 of 236756
INFO:tensorflow:Writing example 150000 of 236756
INFO:tensorflow:Writing example 160000 of 236756
INFO:tensorflow:Writing example 170000 of 236756
INFO:tensorflow:Writing example 180000 of 236756
INFO:tensorflow:Writing example 190000 of 236756
INFO:tensorflow:Writing example 200000 of 236756
INFO:tensorflow:Writing example 210000 of 236756
INFO:tensorflow:Writing example 220000 of 236756
INFO:tensorflow:Writing example 230000 of 236756


In [23]:
tf.logging.info('***** Running training *****')
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 236756
INFO:tensorflow:  Batch size = 32
INFO:tensorflow:  Num steps = 22195


In [32]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=FLAGS.max_seq_length,
    is_training=True,
    drop_remainder=True)

In [35]:
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (32, 128)
INFO:tensorflow:  name = input_mask, shape = (32, 128)
INFO:tensorflow:  name = is_real_example, shape = (32,)
INFO:tensorflow:  name = label_ids, shape = (32,)
INFO:tensorflow:  name = segment_ids, shape = (32, 128)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set 

INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_6/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_6/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorfl

INFO:tensorflow:  name = bert/encoder/layer_11/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = output_weights:0, shape = (2, 768)
INFO:ten

INFO:tensorflow:global_step/sec: 3.01138
INFO:tensorflow:examples/sec: 96.364
INFO:tensorflow:global_step/sec: 3.01138
INFO:tensorflow:examples/sec: 96.3641
INFO:tensorflow:global_step/sec: 3.03883
INFO:tensorflow:examples/sec: 97.2427
INFO:tensorflow:global_step/sec: 2.99333
INFO:tensorflow:examples/sec: 95.7865
INFO:tensorflow:global_step/sec: 2.97555
INFO:tensorflow:examples/sec: 95.2175
INFO:tensorflow:global_step/sec: 3.01137
INFO:tensorflow:examples/sec: 96.3639
INFO:tensorflow:global_step/sec: 2.99334
INFO:tensorflow:examples/sec: 95.787
INFO:tensorflow:global_step/sec: 3.00233
INFO:tensorflow:examples/sec: 96.0747
INFO:tensorflow:global_step/sec: 3.0481
INFO:tensorflow:examples/sec: 97.5392
INFO:tensorflow:global_step/sec: 2.9844
INFO:tensorflow:examples/sec: 95.5008
INFO:tensorflow:global_step/sec: 2.98442
INFO:tensorflow:examples/sec: 95.5014
INFO:tensorflow:global_step/sec: 2.98441
INFO:tensorflow:examples/sec: 95.501
INFO:tensorflow:global_step/sec: 2.98441
INFO:tensorflow:

...

INFO:tensorflow:examples/sec: 96.0747
INFO:tensorflow:global_step/sec: 2.9844
INFO:tensorflow:examples/sec: 95.5007
INFO:tensorflow:global_step/sec: 2.91481
INFO:tensorflow:examples/sec: 93.2741
INFO:tensorflow:global_step/sec: 2.9405
INFO:tensorflow:examples/sec: 94.0961
INFO:tensorflow:global_step/sec: 3.03876
INFO:tensorflow:examples/sec: 97.2402
INFO:tensorflow:global_step/sec: 3.16394
INFO:tensorflow:examples/sec: 101.246
INFO:tensorflow:global_step/sec: 3.02955
INFO:tensorflow:examples/sec: 96.9456
INFO:tensorflow:global_step/sec: 3.16394
INFO:tensorflow:examples/sec: 101.246
INFO:tensorflow:global_step/sec: 3.02955
INFO:tensorflow:examples/sec: 96.9455
INFO:tensorflow:global_step/sec: 3.17389
INFO:tensorflow:examples/sec: 101.565
INFO:tensorflow:global_step/sec: 3.15387
INFO:tensorflow:examples/sec: 100.924
INFO:tensorflow:global_step/sec: 3.184
INFO:tensorflow:examples/sec: 101.888
INFO:tensorflow:global_step/sec: 3.15387
INFO:tensorflow:examples/sec: 100.924
INFO:tensorflow:gl

INFO:tensorflow:global_step/sec: 3.184
INFO:tensorflow:examples/sec: 101.888
INFO:tensorflow:global_step/sec: 3.16392
INFO:tensorflow:examples/sec: 101.246
INFO:tensorflow:global_step/sec: 3.16385
INFO:tensorflow:examples/sec: 101.243
INFO:tensorflow:global_step/sec: 3.1739
INFO:tensorflow:examples/sec: 101.565
INFO:tensorflow:global_step/sec: 3.17382
INFO:tensorflow:examples/sec: 101.562
INFO:tensorflow:global_step/sec: 3.17389
INFO:tensorflow:examples/sec: 101.565
INFO:tensorflow:global_step/sec: 3.16393
INFO:tensorflow:examples/sec: 101.246
INFO:tensorflow:global_step/sec: 3.18392
INFO:tensorflow:examples/sec: 101.886
INFO:tensorflow:global_step/sec: 3.1739
INFO:tensorflow:examples/sec: 101.565
INFO:tensorflow:global_step/sec: 3.184
INFO:tensorflow:examples/sec: 101.888
INFO:tensorflow:global_step/sec: 3.17389
INFO:tensorflow:examples/sec: 101.565
INFO:tensorflow:global_step/sec: 3.1739
INFO:tensorflow:examples/sec: 101.565
INFO:tensorflow:global_step/sec: 3.16393
INFO:tensorflow:ex

INFO:tensorflow:examples/sec: 98.1389
INFO:tensorflow:global_step/sec: 2.98469
INFO:tensorflow:examples/sec: 95.5101
INFO:tensorflow:global_step/sec: 2.9755
INFO:tensorflow:examples/sec: 95.216
INFO:tensorflow:global_step/sec: 2.96669
INFO:tensorflow:examples/sec: 94.9342
INFO:tensorflow:global_step/sec: 3.06679
INFO:tensorflow:examples/sec: 98.1373
INFO:tensorflow:global_step/sec: 3.12423
INFO:tensorflow:examples/sec: 99.9754
INFO:tensorflow:global_step/sec: 3.11457
INFO:tensorflow:examples/sec: 99.6663
INFO:tensorflow:global_step/sec: 3.1243
INFO:tensorflow:examples/sec: 99.9777
INFO:tensorflow:global_step/sec: 3.13416
INFO:tensorflow:examples/sec: 100.293
INFO:tensorflow:global_step/sec: 3.14396
INFO:tensorflow:examples/sec: 100.607
INFO:tensorflow:global_step/sec: 3.11449
INFO:tensorflow:examples/sec: 99.6637
INFO:tensorflow:global_step/sec: 3.12438
INFO:tensorflow:examples/sec: 99.9801
INFO:tensorflow:global_step/sec: 3.1243
INFO:tensorflow:examples/sec: 99.9777
INFO:tensorflow:gl

INFO:tensorflow:examples/sec: 101.243
INFO:tensorflow:global_step/sec: 3.17381
INFO:tensorflow:examples/sec: 101.562
INFO:tensorflow:global_step/sec: 3.17398
INFO:tensorflow:examples/sec: 101.567
INFO:tensorflow:global_step/sec: 3.17381
INFO:tensorflow:examples/sec: 101.562
INFO:tensorflow:global_step/sec: 3.184
INFO:tensorflow:examples/sec: 101.888
INFO:tensorflow:global_step/sec: 3.12999
INFO:tensorflow:examples/sec: 100.16
INFO:tensorflow:global_step/sec: 3.14395
INFO:tensorflow:examples/sec: 100.607
INFO:tensorflow:global_step/sec: 3.03883
INFO:tensorflow:examples/sec: 97.2427
INFO:tensorflow:global_step/sec: 3.00242
INFO:tensorflow:examples/sec: 96.0773
INFO:tensorflow:global_step/sec: 2.97552
INFO:tensorflow:examples/sec: 95.2168
INFO:tensorflow:global_step/sec: 2.9579
INFO:tensorflow:examples/sec: 94.6529
INFO:tensorflow:global_step/sec: 2.95784
INFO:tensorflow:examples/sec: 94.651
INFO:tensorflow:global_step/sec: 3.09528
INFO:tensorflow:examples/sec: 99.049
INFO:tensorflow:glob

INFO:tensorflow:examples/sec: 100.921
INFO:tensorflow:global_step/sec: 3.14402
INFO:tensorflow:examples/sec: 100.609
INFO:tensorflow:global_step/sec: 3.09529
INFO:tensorflow:examples/sec: 99.0492
INFO:tensorflow:global_step/sec: 2.96663
INFO:tensorflow:examples/sec: 94.9321
INFO:tensorflow:global_step/sec: 2.88124
INFO:tensorflow:examples/sec: 92.1998
INFO:tensorflow:global_step/sec: 2.85637
INFO:tensorflow:examples/sec: 91.4038
INFO:tensorflow:global_step/sec: 2.92332
INFO:tensorflow:examples/sec: 93.5463
INFO:tensorflow:global_step/sec: 3.13409
INFO:tensorflow:examples/sec: 100.291
INFO:tensorflow:global_step/sec: 2.99328
INFO:tensorflow:examples/sec: 95.7848
INFO:tensorflow:global_step/sec: 2.95799
INFO:tensorflow:examples/sec: 94.6558
INFO:tensorflow:global_step/sec: 2.89791
INFO:tensorflow:examples/sec: 92.733
INFO:tensorflow:global_step/sec: 2.91477
INFO:tensorflow:examples/sec: 93.2725
INFO:tensorflow:global_step/sec: 2.93168
INFO:tensorflow:examples/sec: 93.8137
INFO:tensorflow

In [38]:
# Evaluatoin
eval_examples = processor.get_dev_examples(dacon_path, 'dev.tsv')
num_actual_eval_examples = len(eval_examples)

eval_file = os.path.join(FLAGS.output_dir, 'eval.tf_record')
file_based_convert_examples_to_features(
    eval_examples, label_list, FLAGS.max_seq_length, tokenizer, openapi_key, eval_file)

INFO:tensorflow:Writing example 0 of 59189
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: dev-1
INFO:tensorflow:tokens: [CLS] 고객/NNG_ 님/XSN_ 의/JKG_ 생 신/NNG_ 을/JKO_ 진 심/NNP_ 으로/JKB_ 축하/NNG_ 드리/VV_ ㅂ 니다/EC_ 행복/NNG_ 하/XSV_ ㄴ/ETM_ 오늘/NNG_ 되/VV_ 시/EP_ 어요/EF_ !/SF_ X X X/SL_ 명 일/NNP_ X X X /NNP_ 팀장/NNG_ 올림 /NNP_ [SEP]
INFO:tensorflow:input_ids: 2 1291 1123 13 1030 445 11 351 7028 28 3503 2120 6921 9404 2056 9 10 1132 57 388 526 524 3047 3047 1496 859 1747 3047 3047 3047 107 3875 9012 107 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0

In [39]:
tf.logging.info("***** Running evaluation *****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(eval_examples), num_actual_eval_examples,
                len(eval_examples) - num_actual_eval_examples)
tf.logging.info("  Batch size = %d", FLAGS.eval_batch_size)

INFO:tensorflow:***** Running evaluation *****
INFO:tensorflow:  Num examples = 59189 (59189 actual, 0 padding)
INFO:tensorflow:  Batch size = 8


In [42]:
eval_steps = None

In [40]:
eval_drop_remainder = True if FLAGS.use_tpu else False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=eval_drop_remainder)

In [43]:
result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30349, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *I

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorfl

In [45]:
# How to check if the code is running on GPU or CPU?

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9684937415421560407
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9106889769
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2933602892538035536
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:09:00.0, compute capability: 7.5"
]


In [48]:
tf.test.is_gpu_available()

True